<a href="https://colab.research.google.com/github/Offliners/writeup/blob/main/HW7/homework7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [1]:
# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Wed May 19 05:02:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
7.71MB [00:00, 121MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 9.1MB/s 
     |████████████████████████████████| 3.3MB 23.1MB/s 
     |████████████████████████████████| 901kB 32.6MB/s 


## Import Packages

In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup
from random import randint

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [5]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 51kB 1.8MB/s 


## Load Model and Tokenizer




 

In [9]:
# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

# from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
# tokenizer = AutoTokenizer.from_pretrained("nyust-eb210/braslab-bert-drcd-384")

# model = AutoModelForQuestionAnswering.from_pretrained("nyust-eb210/braslab-bert-drcd-384")

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
tokenizer = AutoTokenizer.from_pretrained("wptoux/albert-chinese-large-qa")

model = AutoModelForQuestionAnswering.from_pretrained("wptoux/albert-chinese-large-qa")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [10]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [11]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [14]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 50
        self.max_paragraph_len = 300
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 256

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            # mid = (answer_start_token + answer_end_token) // 2
            # paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            if answer_end_token - answer_start_token <= 0:
                paragraph_start = 0
            else:
                pos = randint(answer_start_token, answer_end_token)
                paragraph_start = max(0, min(pos - int(self.max_paragraph_len * (pos - answer_start_token) / (answer_end_token - answer_start_token)), len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
# temp_set = ConcatDataset([train_set, dev_set])
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [18]:
def evaluate(data, output, index):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            if answer.find('[UNK]') != -1:
                countUNK = answer.count('[UNK]')
                flag = 0
                print(f'ID {index} Before : {answer}')
                while answer.find('[UNK]') != -1:
                    window_len = end_index - start_index + 10
                    start = 0
                    end = window_len
                    if answer.find('[UNK]') > 1:
                        target1 = answer[answer.find('[UNK]') - 2]
                    else:
                        target1 = -1
                    if len(answer) - (answer.find('[UNK]') + len('[UNK]')) > 0:
                        target2 = answer[answer.find('[UNK]') + len('[UNK]') + 1]
                    else:
                        target2 = -1
                    count = 0
                    for i in range(len(test_paragraphs[test_questions[index]['paragraph_id']])):
                        if end > len(test_paragraphs[test_questions[index]['paragraph_id']]):
                            final = len(test_paragraphs[test_questions[index]['paragraph_id']])
                        else:
                            final = end + i + 1
                        para = test_paragraphs[test_questions[index]['paragraph_id']][start+i: final]
                        if target1 != -1 and target2 != -1:
                            if para.find(target1) != -1 and para.find(target2) != -1:
                                target = para[para.find(target1) + 1: para.find(target2)]
                                print(f'Target : {target}')
                                answer = answer.replace('[UNK]', target)
                                print(f'ID {index} After  : {answer}')
                                count += 1
                        elif target1 != -1 and target2 == -1:
                            if para.find(target1) != -1 and para.find(target1) != len(para) - 1:
                                target = para[para.find(target1) + 1]
                                print(f'Target : {target}')
                                answer = answer.replace('[UNK]', target)
                                print(f'ID {index} After  : {answer}')
                                count += 1
                        elif target1 == -1 and target2 != -1:
                            if para.find(target2) != -1 and para.find(target2) != 0:
                                target = para[para.find(target2) - 1]
                                print(f'Target After: {target}')
                                answer = answer.replace('[UNK]', target)
                                print(f'ID {index} After  : {answer}')
                                count += 1
                        else:
                            answer = answer.replace('[UNK]', para[answer.find('[UNK]')])
                            flag = 1
                            break
                        if count >= countUNK:
                            break
                        if i == len(test_paragraphs[test_questions[index]['paragraph_id']]) - 1:
                            flag = 1
                    if flag == 1:
                      break
                        
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [19]:
num_epoch = 2
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=total_steps)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

accum_iter = 4 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    for batch_idx, data in enumerate(tqdm(train_loader)):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        with torch.set_grad_enabled(True):
            # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
            # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
            output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

            # Choose the most probable start position / end position
            start_index = torch.argmax(output.start_logits, dim=1)
            end_index = torch.argmax(output.end_logits, dim=1)
            
            # Prediction is correct only if both start_index and end_index are correct
            train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
            train_loss += output.loss
            
            output.loss = output.loss / accum_iter

            if fp16_training:
                accelerator.backward(output.loss)
            else:
                output.loss.backward()
            
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad()
            
            step += 1

            ##### TODO: Apply linear learning rate decay #####
            scheduler.step()

            # Print training loss and accuracy over past logging step
            if step % logging_step == 0:
                print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
                train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, i) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1 | Step 100 | loss = 0.935, acc = 0.657
Epoch 1 | Step 200 | loss = 0.930, acc = 0.652
Epoch 1 | Step 300 | loss = 0.851, acc = 0.671
Epoch 1 | Step 400 | loss = 0.763, acc = 0.680
Epoch 1 | Step 500 | loss = 0.765, acc = 0.704
Epoch 1 | Step 600 | loss = 0.717, acc = 0.722
Epoch 1 | Step 700 | loss = 0.700, acc = 0.707
Epoch 1 | Step 800 | loss = 0.701, acc = 0.724
Epoch 1 | Step 900 | loss = 0.622, acc = 0.739
Epoch 1 | Step 1000 | loss = 0.581, acc = 0.767
Epoch 1 | Step 1100 | loss = 0.622, acc = 0.751
Epoch 1 | Step 1200 | loss = 0.683, acc = 0.736
Epoch 1 | Step 1300 | loss = 0.607, acc = 0.748
Epoch 1 | Step 1400 | loss = 0.590, acc = 0.765
Epoch 1 | Step 1500 | loss = 0.584, acc = 0.765
Epoch 1 | Step 1600 | loss = 0.643, acc = 0.750
Epoch 1 | Step 1700 | loss = 0.569, acc = 0.766
Epoch 1 | Step 1800 | loss = 0.528, acc = 0.781
Epoch 1 | Step 1900 | loss = 0.601, acc = 0.759
Epoch 1 | Step 2000 | loss = 0.503, acc = 0.800
Epoch 1 | Step 2100 | loss = 0.586, acc = 0.770
E

ID 32 Before : 周 王 是 燕 王 同 母 兄 弟 ， 而 朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
ID 67 Before : 祭 天 、 祈 穀 和 祈 雨 的 場 所 。 是 現 存 中 國 古 代 規 模 最 大 、 倫 理 等 級 最 高 的 祭 祀 建 築 群 。 1961 年 ， 天 壇 被 中 華 人 民 共 和 國 國 務 院 公 布 為 第 一 批 全 國 重 點 文 物 保 護 單 位 之 一 。 1998 年 ， 「 北 京 皇 家 祭 壇 [UNK] 天 壇 」 被 列 為 世 界 文 化 遺 產 。 北 京 天 壇 最 初 為 明 永 樂 十 八 年 仿 南 京 城 形 制 而 建 的 天 地 壇 ， 嘉 靖 九 年 實 行 四 郊 分 祀 制 度
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 529 Before : 《 阿 [UNK] 婆 吠 陀 》
ID 716 Before : 姚 [UNK]
ID 826 Before : 1628 年 年 底 ， 當 時 的 福 建 巡 撫 熊 文 燦 再 度 招 安 鄭 芝 龍 ， 成 為 海 防 遊 擊 。 在 鄭 森 六 歲 的 時 候 ， 才 被 接 回 泉 州 府 晉 江 縣 安 平 鎮 居 住 讀 書 ， 該 處 現 為 安 海 成 功 小 學 校 址 。 崇 禎 十 一 年 ， 鄭 森 考 中 秀 才 ， 又 經 考 試 成 為 南 安 縣 二 十 位 [UNK] 膳 生 之 一 。 崇 禎 十 四 年 ， 迎 娶 福 建 泉 州 惠 安 進 士 禮 部 侍 郎 董 [UNK] 先 侄 女 董 友 。 崇 禎 十 七 年 ， 進 入 南 京 國 子 監 深 造 ， 拜 入 江 浙 名 儒 錢 謙 益 門 下 ； 錢 謙 益 為 勉 勵 鄭 成 功 ， 乃 替 他 起 字 大 木 。 崇 禎 十 七 年 三 月 ， 闖 王 李 自 成 攻 破 北 京 ， 崇 禎 帝 自 [UNK] 於 煤 山 ； 隨 後 明 山 海 關 總 兵 吳 三 桂 引 清 兵 入 關 ， 擊 破 李 自 成 ， 進 駐 燕 京 。 崇 禎 十 七 年 三 月 二 十 九 日 ， 京 師 淪 陷 消

Epoch 2 | Step 100 | loss = 0.385, acc = 0.815
Epoch 2 | Step 200 | loss = 0.451, acc = 0.797
Epoch 2 | Step 300 | loss = 0.386, acc = 0.820
Epoch 2 | Step 400 | loss = 0.461, acc = 0.796
Epoch 2 | Step 500 | loss = 0.439, acc = 0.808
Epoch 2 | Step 600 | loss = 0.470, acc = 0.806
Epoch 2 | Step 700 | loss = 0.393, acc = 0.830
Epoch 2 | Step 800 | loss = 0.460, acc = 0.819
Epoch 2 | Step 900 | loss = 0.401, acc = 0.797
Epoch 2 | Step 1000 | loss = 0.463, acc = 0.801
Epoch 2 | Step 1100 | loss = 0.455, acc = 0.814
Epoch 2 | Step 1200 | loss = 0.462, acc = 0.808
Epoch 2 | Step 1300 | loss = 0.387, acc = 0.825
Epoch 2 | Step 1400 | loss = 0.456, acc = 0.797
Epoch 2 | Step 1500 | loss = 0.446, acc = 0.806
Epoch 2 | Step 1600 | loss = 0.400, acc = 0.824
Epoch 2 | Step 1700 | loss = 0.421, acc = 0.827
Epoch 2 | Step 1800 | loss = 0.433, acc = 0.819
Epoch 2 | Step 1900 | loss = 0.422, acc = 0.817
Epoch 2 | Step 2000 | loss = 0.444, acc = 0.811
Epoch 2 | Step 2100 | loss = 0.433, acc = 0.814
E

ID 32 Before : 周 王 是 燕 王 同 母 兄 弟 ， 而 朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 529 Before : 《 阿 [UNK] 婆 吠 陀 》
ID 716 Before : 姚 [UNK]
ID 771 Before : 隋 [UNK] 帝
ID 854 Before : [UNK]
ID 869 Before : 翁 同 [UNK]
Target : 時
ID 869 After  : 翁 同 時
ID 882 Before : 地 狹 人 稠 ， 另 一 項 原 因 為? [SEP] 便 利 商 店 ， 在 日 本 等 地 區 又 稱 為 [UNK] ， 緣 起 於 美 國 公 路 邊 的 加 油 站 附 設 小 店 ， 但 1980 年 代 東 亞 都 市 化 後 在 人 口 密 集 地 區 特 別 流 行 ， 並 擴 散 到 許 多 國 家 的 都 市 ， 通 常 指 規 模 較 小 ， 但 貨 物 種 類 多 元 、 販 售 民 生 相 關 物 資 或 即 食 食 品 的 商 店 ， 其 中 也 包 含 加 油 站 商 店 ， 通 常 位 於 交 通 較 為 便 捷 之 處 ， 以 民 生 服 務 取 勝 ， 便 利 店 有 時 被 當 作 小 型 超 市 。 便 利 商 店 的 開 始 應 是 在 1930 年 ， 美 國 南 方 公 司 於 美 國 達 拉 斯 開 設 了 27 家 圖 騰 商 店 ， 並 於 1946 年 延 長 營 業 時 間 為 早 上 7 點 至 晚 上 11 點 ， 所 以 將 商 店 命 名 為 7 - [UNK] 。 日 本 的 便 利 商 店 為 60 年 代 開 始 發 展 ， 至 今 已 有 相 當 的 規 模 。 全 日 本 的 連 鎖 便 利 商 店 集 團 共 計 十 多 個 ， 其 中 以 7 - [UNK] 、 [UNK] 及 [UNK] 營 收 最 大 ， 消 費 者 常 以 營 業 據 點 和 品 牌 形 象 做 為 偏 好 商 店 的 主 要 選 擇 原 因 。
Target : 「埃歐」或「伊俄」
ID 882 After  

In [ ]:
test_paragraphs[test_questions[49]['paragraph_id']]

In [20]:
# Postprocessing

def postprocessing(result, index):
    if result:
        if result[0] == '，' or result[0] == '。':
            print(f'ID {index} Before : {result}')
            result = result[1:]
            print(f'ID {index} After  : {result}')

        if result[0] == '《' and result[-1] != '》' and result.find('》') == -1:
            print(f'ID {index} Before : {result}')
            result = result + '》'
            print(f'ID {index} After  : {result}')
        
        if result[0] != '《' and result[-1] == '》' and result.find('《') == -1:
            print(f'ID {index} Before : {result}')
            result = '《' + result  
            print(f'ID {index} After  : {result}')
        
    return result

## Testing

In [21]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, i))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
                result[i] = postprocessing(result[i], i)
                f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


ID 26 Before : 拉 丁 文 [UNK]
Target : 中
ID 26 After  : 拉 丁 文 中
ID 38 Before : 有 沒 有 通 過 任 何 示 威 申 請? [SEP] 在 1980 年 代 ， 中 國 公 民 的 遊 行 示 威 集 會 權 利 雖 不 能 說 有 保 障 ， 但 還 在 合 法 狀 態 下 。 1989 年 六 四 天 安 門 事 件 後 ， 全 國 人 大 常 委 會 在 10 月 31 日 緊 急 立 法 ， 出 台 一 部 憲 法 類 法 律 [UNK] 《 中 華 人 民 共 和 國 集 會 遊 行 示 威 法 》 ， 該 法 規 定 ， 舉 行 集 會 、 遊 行 、 示 威 ， 必 須 提 前 五 日 向 主 管 機 關 提 出 申 請 並 獲 得 許 可 。 很 多 人 表 達 了 對 該 法 律 的 憂 慮 ， 認 為 把 這 項 基 本 人 權 變 成 申 請 許 可 制 度 ， 而 非 備 案 制 度 ， 事 實 上 讓 政 府 有 了 審 查 權 限 。 有 些 專 家 直 接 批 評 它 違 反 憲 法 ， 但 由 於 中 華 人 民 共 和 國 憲 法 目 前 不 能 作 為 判 案 依 據 ， 所 以 無 法 進 行 違 憲 審 查 。 事 實 上 由 於 公 安 部 門 對 遊 行 申 請 基 本 上 不 批 准 ， 使 得 中 國 境 內 進 行 的 絕 大 部 分 遊 行 示 威 活 動 都 在 不 合 法 狀 態 下
Target : —
ID 38 After  : 有 沒 有 通 過 任 何 示 威 申 請? [SEP] 在 1980 年 代 ， 中 國 公 民 的 遊 行 示 威 集 會 權 利 雖 不 能 說 有 保 障 ， 但 還 在 合 法 狀 態 下 。 1989 年 六 四 天 安 門 事 件 後 ， 全 國 人 大 常 委 會 在 10 月 31 日 緊 急 立 法 ， 出 台 一 部 憲 法 類 法 律 — 《 中 華 人 民 共 和 國 集 會 遊 行 示 威 法 》 ， 該 法 規 定 ， 舉 行 集 會 、 遊 行 、 示 威 ， 必 須 提 前 五 日 向 主 管 機 關 提 出 申 請 並 獲 得 許 可 。 很 多 人 表 達 了 對 該 

In [22]:
from google.colab import files
files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>